In [2]:
# !git clone https://github.com/cg123/mergekit.git
# !cd mergekit && pip install -q -e .

Cloning into 'mergekit'...
remote: Enumerating objects: 1243, done.
remote: Counting objects: 100% (627/627), done.
remote: Compressing objects: 100% (220/220), done.
remote: Total 1243 (delta 521), reused 423 (delta 405), pack-reused 616
Receiving objects: 100% (1243/1243), 350.25 KiB | 9.22 MiB/s, done.
Resolving deltas: 100% (844/844), done.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
import yaml

yaml_config = """
models:
  - model: mlabonne/NeuralMonarch-7B
    # No parameters necessary for base model
  - model: mlabonne/AlphaMonarch-7B
    parameters:
      density: 0.53
      weight: 0.4
  - model: bardsai/jaskier-7b-dpo-v5.6
    parameters:
      density: 0.53
      weight: 0.3
  - model: macadeliccc/MBX-7B-v3-DPO
    parameters:
      density: 0.53
      weight: 0.3
merge_method: dare_ties
base_model: mlabonne/NeuralMonarch-7B
parameters:
  int8_mask: true
dtype: bfloat16
"""

# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)

In [4]:
# Merge models
!mergekit-yaml config.yaml merge --copy-tokenizer --allow-crimes --out-shard-size 1B --lazy-unpickle

Fetching 10 files:   0%|                                 | 0/10 [00:00<?, ?it/s]

generation_config.json: 100%|███████████████████| 111/111 [00:00<00:00, 542kB/s]

Fetching 10 files:  20%|█████                    | 2/10 [00:02<00:09,  1.20s/it]

tokenizer_config.json: 100%|███████████████████| 971/971 [00:00<00:00, 5.66MB/s]


model.safetensors.index.json: 100%|████████| 23.9k/23.9k [00:00<00:00, 62.2MB/s]


tokenizer.json:   0%|                               | 0.00/1.80M [00:00<?, ?B/s]


special_tokens_map.json: 100%|█████████████████| 552/552 [00:00<00:00, 2.04MB/s]
tokenizer.json: 100%|██████████████████████| 1.80M/1.80M [00:00<00:00, 25.1MB/s]


model-00002-of-00003.safetensors:   0%|             | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|     | 21.0M/5.00G [00:00<00:32, 152MB/s]


model-00001-of-00003.safetensors:   0%|             | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   1%|     | 41.9M/5.00G [00:00<00:29, 165MB/s]



tokenizer.

In [6]:
!pip install -qU huggingface_hub

from huggingface_hub import ModelCard, ModelCardData
from jinja2 import Template
username = "CorticalStack"
MODEL_NAME = "pastiche-crown-clown-7B-dare"


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [7]:
template_text = """
---
license: apache-2.0
tags:
- merge
- mergekit
{%- for model in models %}
- {{ model }}
{%- endfor %}
---

# {{ model_name }}

{{ model_name }} is a DARE merge of the following models using [mergekit](https://github.com/cg123/mergekit):

{%- for model in models %}
* [{{ model }}](https://huggingface.co/{{ model }})
{%- endfor %}

See the paper [Language Models are Super Mario: Absorbing Abilities from Homologous Models as a Free Lunch](https://arxiv.org/abs/2311.03099) for more on the method.

## 🧩 Configuration

```yaml
{{- yaml_config -}}
```
"""

In [8]:
# Create a Jinja template object
jinja_template = Template(template_text.strip())

# Get list of models from config
data = yaml.safe_load(yaml_config)
if "models" in data:
    models = [data["models"][i]["model"] for i in range(len(data["models"])) if "parameters" in data["models"][i]]
elif "parameters" in data:
    models = [data["slices"][0]["sources"][i]["model"] for i in range(len(data["slices"][0]["sources"]))]
elif "slices" in data:
    models = [data["slices"][i]["sources"][0]["model"] for i in range(len(data["slices"]))]
else:
    raise Exception("No models or slices found in yaml config")

# Fill the template
content = jinja_template.render(
    model_name=MODEL_NAME,
    models=models,
    yaml_config=yaml_config,
    username=username,
)

# Save the model card
card = ModelCard(content)
card.save('merge/README.md')

In [9]:
from huggingface_hub import HfApi
HF_TOKEN = ""

# Defined in the secrets tab in Google Colab
api = HfApi(token=HF_TOKEN)

api.create_repo(
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="merge",
)

model-00003-of-00008.safetensors:   0%|          | 0.00/1.97G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Upload 9 LFS files:   0%|          | 0/9 [00:00<?, ?it/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/789M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/CorticalStack/pastiche-crown-clown-7B-dare/commit/3bf41a349169fccc8d050ee9fd87511c4afe6404', commit_message='Upload folder using huggingface_hub', commit_description='', oid='3bf41a349169fccc8d050ee9fd87511c4afe6404', pr_url=None, pr_revision=None, pr_num=None)